# Python（on Google Colab）で遺伝子発現解析（納豆菌編）
遺伝子発現解析をGoogle Colagoratory（通称；Google Colab）上のPythonで試しにやってみた。
全部の遺伝子で遺伝子発現を調べられるトランスクリプトーム（transcriptome）のデータについて、培養条件によって発現に差のある遺伝子をピックアップし、それらが生物学的にどういう機能があるかについて解釈を行っている（あくまで実際の統計を体験してみるのが目的なので、一部 簡略化しているところがある）。（参考までに、同じことをExcelでやった版や、今回扱う納豆菌でなくてシロイヌナズナでやったバージョンもあります）


## Google Colabの利用について
このファイルを自分のGoogle Colaboratory（Google Colab）で開くと編集なりができるようになるはずです。作ったファイルは自分のGoogle Driveに保存されます。
右上にGoogleアカウントのアイコンが出ているはずですが、（私も含めて）複数のGoogle アカウントを持っていると、えてして別のアカウントでGoogle Colabが開いてしまうこともあります。その場合は保存されるファイルもそのアカウントのGoogle Driveに保存されるので、あらかじめ右上の自分のアイコンをクリックして適切なアカウントにしてから作業を開始しましょう。ファイルを行方不明にしないように。。。

## 背景
### 今回 利用するデータ
- GSE109523 ：Transcriptome analysis of Bacillus subtilis NBRC 16449 grown on surface of boiled soybeans under the similar condition to production of Japanese traditional soybean-fermented food "natto" https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE109523
- 日本語訳：Bacillus subtilis（枯草菌・こそうきん）NBRC 16449株における日本の伝統的な大豆発酵食品 納豆の生産と同条件下における茹でた大豆の表面での成長時のトランスクリプトーム（全遺伝子発現）解析
- 今回はこれを加工したデータを用いる ( https://raw.githubusercontent.com/chalkless/lecture/master/biostats/exp_natto/Bsubtilis_exp.for_lec.0613.txt )。あらかじめダウンロードしておく。（右クリックして「名前をつけて保存」）

### 生物学的背景
- あとで書く

### [参考][補足] GEOからの遺伝子発現データの取得
- 今回は、すぐにPythonやExcelで開けるように加工済のデータになっているが、元々の配布されているデータは別形式（＋実験データだけでなく実験条件の情報付き）なのでそのままでは使えない。
- 後で書く。Excelの場合にしっかり書いたのでそちらを参考に

## 遺伝子発現データの統計的な解析
### データをアップロードする
- いきなりつまずく人が多発する鬼門ポイントです。。。
- 画面左端のフォルダーマーク（📁 ←こんな感じの）をクリックする
- と、画面左側にファイルのリストが現れる。広くなったファイルのリストエリアにデータのファイルをドラッグするとGoogle Colabから見える形でファイルがアップされる
　  - 「注: アップロードしたファイルはランタイムのリサイクル時に削除されます。」とメッセージが出るが気にしない。これの意味は、**ブラウザのウインドウを閉じてしまったりすると覚えていた（リストとして並んでいた）データファイルが消すからごめんね**、ということ。
    - しばらく放っておくとえてしてアップしたデータのファイルが消えている（このColabのドキュメントは無事）
    - データのファイルが消えてしまったら再度アップすればよい。
    - 途中までの計算結果も消えてしまったりするので、それも再計算する
    



### データを読み込む

In [ ]:
import pandas as pd
df = pd.read_csv('Bsubtilis_exp.for_lec.0613.txt', sep='\t', index_col=0)
df.head()

- `import pandas as pd`：統計用のライブラリpandasを呼んで、pdという名前で呼び出せるようにしています。早い話がおまじないです。（＝なんかよくわからんけど書いとけ）
- `df = pd.read_csv(...)`：pdとして呼び出したpandasのread_csvメソッドでファイルを読み込んでデータフレームdfに格納した。（＝呪文をたくさん知っている大魔神pdを呼び出して、read_csvという呪文でファイルを読み込んで、colab上にdfという表を作ってもらった）
- `df.head()`：データフレームdfの最初5行を表示（表dfに最初の5行を表示する呪文headをかけた → ら、5行分が表示された）
- [補足] `pd.read_csv(...)`の...について
  - `sep='\t'`：区切り文字（*sep*arator）はタブ（便宜的に\tで表す）。別の例で、カンマ区切り（csv形式）なら\tを , にする。
  - `index_col=0`：今回の場合は最初の列を各行のラベル（=行の名前）として扱っているので、これを追記している。indexは目次。colは列（column）のこと。えてしてプログラミングでは数字は0から数え始めるので、1でなく0を値として与えている。
  - 今回は1行目が各列のラベルになっているので特に書いていないが、1行目からデータが書かれている場合は（列ラベルがない場合は）、`header=None`を追記する。もしくは、今回の場合、1行目がヘッダであることを`header=1`と明記してもよい。（index_colは0始まりなのにheaderは1始まりなのは納得いかないと思うが、私も説明できないので納得いかない）

- [参考] 自分でもっといろいろ試してやろうと思っている人向けの解説
  - 今回、データを統計ライブラリのpandasを使って読み込んでいます。
  - これはデータを表全体として読み込んでいるからです。
  - 普通、Pythonで何か処理をしようとすると、ファイルを1行ずつ読み込んで処理することが多い気もしますが、その際は以下のように書いたりします。

In [ ]:
# このブロックは参考までの紹介なので実行しなくていい
import csv

file = open('Bsubtilis_exp.for_lec.0613.txt')
reader = csv.reader(file)

for row in reader:
  comment = "この行を書き換えて何か処理したいことを書く"

### 改めてデータを眺めてみる

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

- 全データ（df）の最初の方と最後の方を3行ずつ表示してみた。
- 行方向に見ると（たとえばgene0）、P05648というIDで、「chromosomal replication initiator protein DnaA」という名前の遺伝子で、液体培地で培養した時の遺伝子の発現量が834.1と576.6、茹で大豆の表面で培養した時の遺伝子の発現量が312.9と295.5、寒天培地で（略）となっている。
- 列方向に見ると、液体培地で培養した時の実験その1（liquid_1）では、遺伝子gene0の発現量が834.1、遺伝子gene1の発現量が1055.4、（中略）、遺伝子gene4594の発現量が157.1となっている。

### データの分布を、代表する値を計算することで調べてみる
- データの列には遺伝子の発現量が数値で入っている
- ちらちら眺めると0に近い値から千まで数値があるような
- そこで各列のデータの様子を、代表的な値を計算することで概観してみましょう
- たとえば平均値は`mean()`というメソッド（命令・関数）で求めることができます

In [ ]:
mean_liq1 = df['liquid_1'].mean()
print(mean_liq1)

- `mean_liq1 = df['liquid_1'].mean()`：データフレーム（＝表）dfのliquid_1列に対してmeanという関数で処理しています。
- 他の行も同様に計算できます

In [ ]:
mean_liq2 = df['liquid_2'].mean()
print(mean_liq1)

mean_bean1 = df['bean_1'].mean()
print(mean_bean1)

# 上の記述をコピペしてbean_2についても平均を求めてみましょう

- とは言いながら、列名を指定しないと、各データの行でよしなに平均値を一発で計算してくれます。。。（エラーが出ますが、これは遺伝子名などの数値以外の列が入っているのに気にせずに処理しているからです）

In [ ]:
df.mean()

同様に、maxやmin、medianメソッドで最大値、最小値、中央値を求められます。

In [ ]:
df.max()

In [ ]:
df.min()

In [ ]:
df.median()

- と、くどくどと計算してきましたが、pandasではこういった代表的な値をまとめて計算してくれるdescribeメソッドがあるのでこれで一撃で表示させることができます。

In [ ]:
print(df.describe())

これを眺めるに（今回、対象にするのはliquidとbeanですが）、どの列もだいたい代表する統計値が同じくらいの値をとっているので、特に処理（＝正規化）せずとも先に進んでよさそう、ということがわかる

### データの分布をグラフを描いて直接的に確認してみる
- 上では、データについて代表する値を計算することでデータの概観を試みたが、グラフを描くことでもっと直接的にデータの分布を確認することができる。
- 今回の場合は、データの範囲を区切って、そこに入るデータの個数をカウントしてグラフにするヒストグラム（histgram、度数分布）を描いてみる。

In [ ]:
df.plot.hist(y='liquid_1')

- なんかえらい0に偏ったグラフなんですが、、、
- こうやってある桁だけに偏っていたりする場合は、対数をとるとよい場合もあります。
- 後でしっかりやるんですが、ぴろっと対数をとると、以下の感じです

In [ ]:
import numpy as np      # ここの行は例によって呪文

# liquid_1列について対数をとってもとのdfに1列追加している
df['log_liquid1'] = np.log2(df['liquid_1'])

# ヒストグラムを描く
df.plot.hist(y='log_liquid1', bins=30)     # binsはグラフの本数のイメージ

遺伝子発現のデータは対数をとると、左右対称のグラフになることが知られています（正規分布という名前でモデル化できます（＝近似的な関数で記述できる））

- liquid_1とliquid_2は、同じ実験を2回やって実験誤差を小さくしようしたデータ。なので、liquid_1の値とliquid_2の値は同じなのが理想。
- liquid_1とbean_1は実験の条件（この場合は培養条件）が違う。なので、**培養の条件（液体か固体か? 養分の差?）と関連のある遺伝子がその条件だけで増えていることが想像できる。**（なのでliquid_1とbean_1で値が違うものがいるはずだし、それがおもしろい）
- というあたりを、散布図（scatter plot）というグラフを描いて確認してみる。

In [ ]:
df.plot.scatter(x='liquid_1', y='liquid_2')
df.plot.scatter(x='liquid_1', y='bean_1')

- なんか、0周辺とか特に、liquidどうしだとデータがy=xのグラフに載っていそうだけれども、なんかビミョー。
- これも対数をとるか...

In [ ]:
# これは1回呼び出せばよいので今回は不要
# import numpy as np
# というかいまさらだけれども、# を書くとそれ以降はコメントとして扱われるので処理されない（=人間が読む用として何もしない）

df['log_liquid2'] = np.log2(df['liquid_2'])
df['log_bean1'] = np.log2(df['bean_1'])    

In [ ]:
df.plot.scatter(x='log_liquid1', y='log_liquid2')
df.plot.scatter(x='log_liquid1', y='log_bean1')

- やはり実験条件が違うと遺伝子の発現が違う感じがする。
- liquid_1 vs bean_1で（実際は対数をとっているけれども）
  - グラフの右下寄りに出ている点（データ）：liquidの発現量は大きいが、beanの発現量は小さい＝液体中で生きるのに必要な遺伝子 or 茹で大豆の表面で生きるのにいらない遺伝子
  - グラフの左上寄りに出ている点：liquidの発現量は小さいが、beanの発現量は大きい＝液体中で生きるのにいらない遺伝子 or 茹で大豆の表面で生きるのに必要な遺伝子
  - y=xの（心の目で見える）グラフ近辺にある点：液体も茹で大豆の表面も、培養条件と発現量（遺伝子の必要度）に関係のない遺伝子

- では、実際に液体や茹で大豆の表面での生育と関係の深そうな遺伝子を調べてみて、どういう機能を持っているか、確認してみましょう。
- 今回は茹で大豆の表面で遺伝子発現が大きいデータ（グラフで言うと左上の方）の遺伝子について注目してみます。

## 2つの状態での発現量を比較する
### 2回の実験の平均をとる
- 今回は2回ずつ実験しているが、平均をとって実験の誤差を補正することとする。
  - 先ほど求めた平均は、1列（=1実験）をとりだして、すべての値の平均をとる（縦方向の）平均だったが、今回は、1行（=1遺伝子）を取り出して2回の実験の値で（横方向に）平均をとる（そして、それを全部の遺伝子分 操作を繰り返す）
  

In [ ]:
# liquid_1,2, bean_1,2 について、行方向の平均をとって表dfに追加
df['mean_liquid'] = df[['liquid_1', 'liquid_2']].mean(axis=1)
df['mean_bean'] = df[['bean_1', 'bean_2']].mean(axis=1)
df.head(3)

- 平均をとるのは今回は`mean()`メソッド
- `axis=1`は、行方向の意味。列方向だと`axis=0`で、だいたい列方向だから`axis=0`を指定していることになっていて、で、省略可能なので普段は`mean()`としか見えていないだけ

### 平均をとった発現値の対数をとる
- 今、計算した平均をとった発現値に対して対数を計算する。
  - 桁数を圧縮するため
  - 対数をとると正規分布のモデルにフィットするため
  - この後、発現比を計算するので、それを計算しやすく、見やすくするため

In [ ]:
df['log_liquid'] = np.log2(df['mean_liquid'])
df['log_bean'] = np.log2(df['mean_bean'])
df.head(3)

- 別に底は2でも10でもいいのだが、マイクロアレイなどの発現解析の時は、伝統的に「発現比2倍以上」というデータの絞り込みをしていたため、わかりやすくするために底は2を用いている。（というか、そういうこともわかりやすくするために対数をとるのである）

### 2つの状態の発現の比を計算する
- 忘れかけているかもですが、今やろうとしていることは「今回は茹で大豆の表面で遺伝子発現が大きいデータ（グラフで言うと左上の方）の遺伝子について注目してみます。」です。
- なので、茹で大豆の表面で多く発現している遺伝子を抽出します。
- ということで今、計算した液体培地と茹で大豆の表面の2状態で、発現の比を計算します。
- 今回やりたいのは、元のデータが何倍の発現量差があるかなので、対数の比をとるのでなく、比をとった後で対数をとる
- 覚えていますか：log(A/B) = log(A) - log(B)

In [ ]:
df['diff'] = df['log_bean'] - df['log_liquid']
df.head(3)

- diff 列のデータの意味を振り返ってみる
  - `diff = 2` ：log(diff)=2 だから2^2で4倍の発現差がある。この場合、beanの発現が高い
  - `diff = -2`：log(diff)=-2 だから1/(2^2)で1/4倍の発現差がある。この場合、beanの発現が1/4だから、liquidの方が4倍発現が高い
  - `diff = 0`：log(diff)=0 ということ。つまり、2^0=1。ようするに、beanとliquidで発現量が同じということ

- 試しにdiff列で「代表する値」を計算してみる
  - 特に平均値（mean）と中央値（50%）はほぼ0のはず
  - 今回はちょっとずれているけどｗ

In [ ]:
df['diff'].describe()

- 実際のデータの分布をヒストグラムを描いて確認してみる

In [ ]:
df['diff'].plot.hist()

- 柱の間隔が大きくて大雑把なグラフなのでもう少し間隔を短くしてみる

In [ ]:
df['diff'].plot.hist(bins=30)

- 本当は左右対称のグラフになるはずなんですけどね（実際のデータだからずれることもある、ということにしておこう...）

- さきほど、データの分布でも確認しているが、（diffをとる前の）液体培地と茹で大豆の表面について、実際のデータの分布をscatter plotで確認してみる。

In [ ]:
df.plot.scatter(x='log_liquid', y='log_bean')

## 発現に差のある遺伝子の機能解析
- **覚えていますか**：[解析の目標] 納豆菌の培養条件の違い（今回は液体培地とゆで大豆の表面）でどんな機能の遺伝子が特異的に（=その状態のみで）働いているか?（または働いていないか?）を解析する。

### 発現に差のあるデータの絞り込み
- beanで発現の高い遺伝子のみを抽出してみる。今回は試しに８倍以上にしてみる
  - 8 という数字は適当です。
  - 適当というのは、まぁ、こんなもんかな、ってことです。
  - どうしてまぁ、こんなもんかなって思うかは、試しに絞り込んでみて、（全体が数千くらいに対して）いくつくらいに絞り込めるかとか、結果を見てみたらきれいになったとか、信頼度がどのくらいかとか、そういうので決めているだけで、統計的にかっちり基準があるわけではありません。
  - またあとでこのあたりは出てきます
- とりあえず、フィルターするとデータの個数が何個になるか気になるのでその用意をすることとする。

In [ ]:
df.query("diff >= 3")

209個かー。ちょっと多いかなー。16倍（対数をとっているから4より大きい）にしてみるかー

In [ ]:
df.query("diff >= 4")

- 112個になった。まぁこんなもんか。
- 今回はこのくらいで許してやろう
- 結果的に対数で4（実データで16倍）になったが、「このくらいで」というのは人間が見れるくらいの数くらいのイメージ。別に3でも5でも10でもよい。また別基準で上位50件、上位100件としてもよい（ただ だいたい何倍くらいになっているかは確認した方がいい気はする）。

## 遺伝子機能解析
- 出てきた遺伝子を眺めてもよくわからないし、それらが生体の中でどういう機能を持っているか分からないので専門のサイトに問い合わせる。
- 今回はDAVID (DAVID Functional Annotation Bioinformatics Microarray Analysis https://david.ncifcrf.gov/) のサイトから情報を取得する。
-　DAVIDは（遺伝子機能による）エンリッチメント解析ということをやるサイトです。ランダムに選んだ遺伝子セットに対して、この遺伝子セットは顕著にこの機能が多いですね、みたいなものを可視化してくれます。

この後、Protein_IDの列を抽出してDAVIDというウェブサービスに問い合わせ、結果を取得する

In [ ]:
# 4倍以上のものを抽出
df_ext = df.query("diff >= 4")

#　Protein_IDの列だけ抜き出してリスト（Python用語：要素を並べたもの）にする
list_id = df_ext['Protein_ID'].tolist()
print(list_id)   #　確認

#　"-" は処理されないので、リストから除外する
list_out = [ i for i in list_id if not i == "-"]
print(list_out)  #　確認

#　要素数を確認
len(list_out)

83まで減ってしまった。。。まぁいいか。これをURLにしてDAVIDに投げる

メモ書き： https://david.ncifcrf.gov/content.jsp?file=DAVID_API.html を参考にして、URLを組み上げて、DAVIDのサイトにアクセスしてデータを取得する（予定）
- https://david.ncifcrf.gov/api.jsp?type=xxxxx&ids=XXXXX,XXXXX,XXXXXX,&tool=xxxx&annot=xxxxx,xxxxxx,xxxxx の形式


In [ ]:
str_list = ','.join(list_out)
print(str_list)

In [ ]:
url = "https://david.ncifcrf.gov/api.jsp?type=uniprot_accession&ids=" + str_list + "&tool=chartReport&annot=GOTERM_BP_ALL"
print(url)

とりあえず上のURLをクリックするとブラウザで開いて結果が見られる

-　よく使うのは Gene Ontologyという生物的機能を集めたキーワード集から用語を各遺伝子にあてはめた（ラベル付けした）情報（機能アノテーションという。アノテーションは注釈付けの意味）
  -　Ontologyというのは階層型の用語集
  -　階層型というのは 人体 > 泌尿器系 > 腎臓 > 糸球体 のようなこと
-　今回はGOTERM_BP_ALLを見ている。
  -　BPは biological process。生物学的な挙動（そのままw）
  -　teichoic acidやcell wall （細胞壁）の単語がチラホラ
  -　もともとの2群の違いは細胞壁があるかないか
  -　いいじゃないか 
-　本当はGO_CC（Cellular Component：細胞のパーツ）やGO_MF（Molecular Function）でもいろいろ出てほしいのだが...

##　まとめと発展
- Excelで統計の手法で遺伝子を絞り込み、機能解析をしてみた
- よくR言語での〜というのがあるが、いまはやりのPythonを試してみた。（Excelでもこのくらいならできる）
-　今回は、beanで発現の高い遺伝子について解析したが、もちろんliquidで発現の高い遺伝子についても同様の解析をやってみたりする。
-　さらに、今回は発現の高い遺伝子「セット」で解析をしたが、そこに含まれる実際の遺伝子は何か、というのをマイクロアレイのプローブIDから遺伝子名に直して眺めたりするわけである。
-　GEOのサイトにあるSOFT formatted family file(s)には対応表が載っているので、あらかじめそれをExcelででも列追加して解析を始めるか、あとで個別に眺めるかするわけである
-　さらに今回は、液体培地と茹でた大豆の表面いう2群だったが、他の例だと「通常と薬剤処理の細胞」、「通常期と繁殖期の魚」、「淡水型と海水型の魚」、「春型と夏型のチョウ」、「タバコを吸う人と吸わない人」などなどの前提条件と、条件の違いで遺伝子発現がどう変化するかというような研究が考えられるわけだ。
-　生物統計というと、ついついどんな統計手法で、と統計の方に目がいってしまうが、**生物学のゴールは統計処理でなくて、そこで得られた遺伝子セットを生物学的に解釈する（そして最初の実験条件と照らし合わせる）、というところなのをお忘れなく。**

以下は検討中

In [ ]:
import requests

url_base = "https://david.ncifcrf.gov/api.jsp"
param_david = {
    "type" : "uniprot_accession",
    "ids" : str_list,
    "tool" : "chartReport",
    "annot" : "GOTERM_ALL"
}
r = requests.get(url_base, params=param_david)

r.content